In [12]:
#referenced Dr. Gibson's sample code, https://www.freecodecamp.org/news/how-to-create-a-gui-quiz-application-using-tkinter-and-open-trivia-db/


import random
from tkinter import *
import requests

url = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
get_request = requests.get(url).json()

root = Tk()
root.title("Trivia")
root.geometry("1000x800")

canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.place(relx=.5, rely=.5,anchor= CENTER) #anchors the canvas in the center


game_display = canvas.create_text(200, 75, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

user_answer = StringVar()
user_answer.set("")

score = 0

question_count = 0

global num_correct

num_correct = 0

score_label = Label(root, text=f"Score: {score}", font=("arial", 20, "bold"))


final_score_label = Label(root, text="", font=("arial", 20, "bold"))

def generate_qlist(json):
    question_list = []
    for i in json:
        i = i["question"]
        question_list.append(i)
    return question_list

def gen_question(list_):
    choice = random.choice(list_)
    return choice

def generate_answers(json, question): 
    option_list = []
    for i in get_request:
        if i["question"] == question:
            cor = i["correctAnswer"]
            option_list.append(cor)
            for incor in i["incorrectAnswers"]:
                option_list.append(incor)
    random.shuffle(option_list)
    return option_list

def ask_question(question, answers):
    canvas.itemconfig(game_display, text=question, width=300)
    
def gen_radiobuttons(answers):
    y_pos = 100
    for i in answers:
        rad_btn = Radiobutton(canvas, text=i, variable=user_answer, command=eval_score, value=i, font=('Helvetica 10 bold'))
        rad_btn.place(x=50, y=y_pos)
        y_pos+=20
    
def eval_score():
    answer = user_answer.get()
    for i in get_request:
        if i["question"] == current_question:
            if i["correctAnswer"] == answer:
                feedback["text"] = "Correct!"
                feedback.place(x=400, y=500)
                global score
                global num_correct
                num_correct = num_correct + 1
                score += 10
                score_label["text"] = f"Score: {score}"  # update score label
            else:
                feedback["text"] = "incorrect :("
                feedback.place(x=400, y=500)


                
def main(): 
    q_list = generate_qlist(get_request)
    global current_question
    current_question = gen_question(q_list)
    ans_list = generate_answers(get_request, current_question)
    ask_question(current_question, ans_list)
    gen_radiobuttons(ans_list)

def next_question():
    global current_question
    global score
    global question_count
    question_count += 1
    if question_count <= len(get_request):
        q_list = generate_qlist(get_request)
        current_question = gen_question(q_list)
        ans_list = generate_answers(get_request, current_question)
        ask_question(current_question, ans_list)
        gen_radiobuttons(ans_list)
        feedback["text"] = ""
    else:
        canvas.itemconfig(game_display, text="")
        final_score_label["text"] = f"Final score: {score}/{question_count*10}"
        final_score_label.place(relx=.5, rely=.5, anchor=CENTER)

next_button = Button(root, text="Next", command=next_question, font=("arial", 15, "bold"), bg="light blue")
next_button.place(x=800, y=400)

score_label.place(x=800, y=50)

main()
root.mainloop()


#the most i can get to work on the next question function has the fault of not deleting the old one. I tried a function that erases the old interface everytime, but i had trouble getting it to only delete the answers, it was deleting other things as well. I am having difficulty adding a part to the next question function that deletes the one before
#I tried to edit the next question function to add a counting variable that checks if there are more next questions then questions to be answered, and when there are, you get a final score. The issue with this code is that it does not delete the question we already completed from the old list, so questions can still repeat, meaning other questions will never be seen